In [1]:
! nvidia-smi
! pip install optuna xgboost==1.6.1 pytorch-tabnet

Fri Feb 17 14:44:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
# from xgboost import XGBRegressor
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error,r2_score
import optuna as opt
import joblib
import random
import os

In [4]:
def make_save_cv_model(i,model_name,model,optim,mse_loss,trial_data,output_path="./"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/mse_loss.txt"),"w+") as file:file.write(f"{mse_loss}")
        joblib.dump(trial_data, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_trial_data.z"))
    else:
        os.mkdir(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/mse_loss.txt"),"w+") as file:file.write(f"{mse_loss}")
        joblib.dump(trial_data, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_trial_data.z"))

In [5]:
def save_cv_model(i,model_name,model,optim,mse_loss,output_path="./"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/losses_{fold}.txt"),"w+") as file:file.write(f"{mse_loss}")
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/losses_{fold}.txt"),"w+") as file:file.write(f"{mse_loss}")

In [6]:
def get_trial_data(trial) -> list:
  ''' This function takes the trial objects and returns the dictionary containing the trial details for plotting and comparing purposes '''
  trial_data = trial.get_trials()
  value_dict = {}
  for i in trial_data:
    print(i.params)
    value_dict[i.number] = {"params": i.params , "rmse": i.values}
    print(f"{i.number} : {i.values}")
  return value_dict

In [7]:
def save_optuna_plots(study,dirname):
    optim_hist = opt.visualization.plot_optimization_history(study)
    intermediate = opt.visualization.plot_intermediate_values(study)
    parallel = opt.visualization.plot_parallel_coordinate(study)
    plot_slice = opt.visualization.plot_slice(study)
    name_lists = ["optim_hist","intermediate","parallel", "plot_slice"]
    plot_lists = [optim_hist,intermediate,parallel, plot_slice]
    for name,plot in zip(name_lists,plot_lists):
        if os.path.exists(f"./{dirname}"):
            print("getting into if block")
            plot.write_image(f"./{dirname}/{name}.jpg",width=2, height=2)
        else:
            print("getting into else block")
            os.mkdir(f"./{dirname}")
            plot.write_image(f"./{dirname}/{name}.jpg",width=2, height=2)

In [8]:
def train_trial(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    print(y.shape)
    x = sc_df.drop([tar_col],axis=1)
    print(x.shape)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetRegressor(n_d=trial.suggest_int("n_d", 8, 64),
                                    n_a =trial.suggest_int("n_a", 8, 64),
                                    n_steps = trial.suggest_int("n_steps",3,10),
                                    gamma =trial.suggest_float("gamma", 1.0, 2.0),
                                    n_independent = trial.suggest_int("n_independent",1,5),
                                    n_shared = trial.suggest_int("n_shared",1,5),
                                    momentum = trial.suggest_float("momentum", 0.01, 0.4),
                                    optimizer_fn = torch.optim.Adam,
                                    # scheduler_fn = torch.optim.lr_scheduler,
                                    # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                                    verbose = verbose,
                                    device_name = "cuda" if torch.cuda.is_available() else "cpu"
                                    )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index].to_numpy(dtype=np.float64), y.iloc[test_index].to_numpy(np.float64)
      # Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(X_train.shape)
      print(Y_train.shape)
      print(X_test.shape)
      print(Y_test.shape)
      Y_train = Y_train.reshape(-1,1)
      clf.fit(X_train, Y_train)
      Y_pred = clf.predict(X_test)
      mse_error = mean_squared_error(Y_pred, Y_test, squared=False) 
      return mse_error

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='minimize')
    study.optimize(objective, n_trials=optim_trial)
    save_optuna_plots(study,"optuna_plots")
    best_params = study.best_params
    trial_data = get_trial_data(study)
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetRegressor(**study.best_params,device_name= "cuda" if torch.cuda.is_available() else "cpu")
    Y_train = Y_train.reshape(-1,1)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    error = {"mse_error" : mean_squared_error(Y_pred, Y_test, squared=False),
    "mae_error" : mean_absolute_error(Y_pred,Y_test),
    "rmse_error" : mean_squared_error(Y_pred, Y_test),
    "r2_score" : r2_score(Y_pred,Y_test) }
    # try:
    print("[++] Saving the model and parameters in corresponding directories")
    make_save_cv_model(fold,model_name,clf_model,optim,mse_loss=error,trial_data=trial_data)
    return trial_data,best_params

In [9]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,k_folds,best_params,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    print(y.shape)
    x = sc_df.drop([tar_col],axis=1)
    print(x.shape)
    model_name = model_name 
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetRegressor(**best_params,device_name= "cuda" if torch.cuda.is_available() else "cpu")
    Y_train = Y_train.reshape(-1,1)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    error = {
    "mse_error" : mean_squared_error(Y_pred, Y_test, squared=False),
    "mae_error" : mean_absolute_error(Y_pred,Y_test),
    "rmse_error" : mean_squared_error(Y_pred, Y_test),
    "r2_score" : r2_score(Y_pred,Y_test) }
    # try:
    print("[++] Saving the model and parameters in corresponding directories")
    save_cv_model(fold,model_name,clf_model,optim,mse_loss=error)

In [10]:
use_df = pd.read_csv("../input/perov-scaled-data/scaled_trainable.csv")
tar_col = "JV_default_PCE_numeric"
model_name = "tabnet_reg"
fold_dict = joblib.load("../input/perov-fold-data/fold_data_export.z")
optim = "adam"
k_folds = 20
trial_fold = random.choice([x for x in range(k_folds)])
num_trials = 15

trial_data,best_params = train_trial(fold_dict = fold_dict,
      fold = trial_fold,
      k_folds=k_folds,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim = optim,
      optim_trial = num_trials)
for key,value in trial_data.items():
  print(f"{key}: {value['rmse']}")
print(f"[++] Ended the training process for fold {trial_fold}")


main_folds = [x for x in range(k_folds)]
for fold in main_folds:
    train(fold_dict = fold_dict,
          fold = fold,
          k_folds=k_folds,
          model_name=model_name,
          sc_df=use_df,
          tar_col=tar_col,
          optim = optim,
          best_params = best_params)
    print(f"[++] Ended the training process for fold {fold}")

[I 2023-02-17 14:45:27,105] A new study created in memory with name: no-name-750da9dc-61e8-431f-ab01-6dc907d6a244


(46820,)
(46820, 103)
Starting optimization for fold : [18/20]
(44479, 103)
(44479,)
(2341, 103)
(2341,)


/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 135.87072|  0:00:05s
epoch 1  | loss: 74.57016|  0:00:10s
epoch 2  | loss: 58.03098|  0:00:15s
epoch 3  | loss: 31.13192|  0:00:20s
epoch 4  | loss: 24.87486|  0:00:26s
epoch 5  | loss: 20.03892|  0:00:31s
epoch 6  | loss: 17.89666|  0:00:36s
epoch 7  | loss: 16.29748|  0:00:40s
epoch 8  | loss: 14.77788|  0:00:45s
epoch 9  | loss: 14.31684|  0:00:50s
epoch 10 | loss: 13.65709|  0:00:56s
epoch 11 | loss: 12.87542|  0:01:00s
epoch 12 | loss: 12.6173 |  0:01:05s
epoch 13 | loss: 12.11638|  0:01:10s
epoch 14 | loss: 11.87871|  0:01:15s
epoch 15 | loss: 11.67392|  0:01:20s
epoch 16 | loss: 11.17395|  0:01:25s
epoch 17 | loss: 10.95202|  0:01:30s
epoch 18 | loss: 10.62529|  0:01:35s
epoch 19 | loss: 10.41418|  0:01:40s
epoch 20 | loss: 10.18635|  0:01:44s
epoch 21 | loss: 10.22864|  0:01:49s
epoch 22 | loss: 9.87715 |  0:01:54s
epoch 23 | loss: 9.87845 |  0:01:59s
epoch 24 | loss: 9.43164 |  0:02:04s
epoch 25 | loss: 9.6484  |  0:02:09s
epoch 26 | loss: 9.32056 |  0:02:14s


[I 2023-02-17 14:53:48,833] Trial 0 finished with value: 2.7376944614398324 and parameters: {'n_d': 46, 'n_a': 59, 'n_steps': 10, 'gamma': 1.6768572281509007, 'n_independent': 4, 'n_shared': 1, 'momentum': 0.31161473810203194}. Best is trial 0 with value: 2.7376944614398324.
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 161.31129|  0:00:05s
epoch 1  | loss: 55.40113|  0:00:11s
epoch 2  | loss: 47.39788|  0:00:17s
epoch 3  | loss: 45.95607|  0:00:22s
epoch 4  | loss: 41.01898|  0:00:29s
epoch 5  | loss: 24.51948|  0:00:34s
epoch 6  | loss: 17.47437|  0:00:40s
epoch 7  | loss: 14.57192|  0:00:46s
epoch 8  | loss: 13.43216|  0:00:52s
epoch 9  | loss: 13.11074|  0:00:57s
epoch 10 | loss: 12.55298|  0:01:04s
epoch 11 | loss: 11.714  |  0:01:09s
epoch 12 | loss: 11.76911|  0:01:15s
epoch 13 | loss: 11.5087 |  0:01:21s
epoch 14 | loss: 11.73956|  0:01:27s
epoch 15 | loss: 11.21534|  0:01:33s
epoch 16 | loss: 10.62396|  0:01:39s
epoch 17 | loss: 10.32216|  0:01:44s
epoch 18 | loss: 10.07262|  0:01:50s
epoch 19 | loss: 9.98665 |  0:01:56s
epoch 20 | loss: 9.48407 |  0:02:02s
epoch 21 | loss: 9.47992 |  0:02:08s
epoch 22 | loss: 9.70542 |  0:02:13s
epoch 23 | loss: 9.39728 |  0:02:19s
epoch 24 | loss: 9.13898 |  0:02:25s
epoch 25 | loss: 9.57826 |  0:02:31s
epoch 26 | loss: 9.00931 |  0:02:37s


/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning:

Device used : cuda

/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning:

No early stopping will be performed, last training weights will be used.



epoch 0  | loss: 169.46079|  0:00:06s
epoch 1  | loss: 50.39759|  0:00:11s
epoch 2  | loss: 42.77008|  0:00:17s
epoch 3  | loss: 44.79586|  0:00:23s
epoch 4  | loss: 35.93852|  0:00:28s
epoch 5  | loss: 31.94879|  0:00:34s
epoch 6  | loss: 23.02367|  0:00:40s
epoch 7  | loss: 17.23494|  0:00:46s
epoch 8  | loss: 15.34003|  0:00:52s
epoch 9  | loss: 14.3053 |  0:00:58s
epoch 10 | loss: 13.49971|  0:01:03s
epoch 11 | loss: 13.18831|  0:01:10s
epoch 12 | loss: 12.68855|  0:01:15s
epoch 13 | loss: 12.0029 |  0:01:21s
epoch 14 | loss: 11.93382|  0:01:26s
epoch 15 | loss: 11.61834|  0:01:32s
epoch 16 | loss: 11.19638|  0:01:38s
epoch 17 | loss: 10.74082|  0:01:44s
epoch 18 | loss: 10.58771|  0:01:50s
epoch 19 | loss: 10.55961|  0:01:55s
epoch 20 | loss: 9.99132 |  0:02:01s
epoch 21 | loss: 10.11291|  0:02:07s
epoch 22 | loss: 9.52019 |  0:02:13s
epoch 23 | loss: 9.74017 |  0:02:19s
epoch 24 | loss: 9.32249 |  0:02:25s
epoch 25 | loss: 9.42131 |  0:02:30s
epoch 26 | loss: 9.01513 |  0:02:36s


/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning:

Device used : cuda

/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning:

No early stopping will be performed, last training weights will be used.



epoch 0  | loss: 167.31278|  0:00:05s
epoch 1  | loss: 59.87645|  0:00:11s
epoch 2  | loss: 58.13297|  0:00:17s
epoch 3  | loss: 58.43914|  0:00:22s
epoch 4  | loss: 39.34564|  0:00:28s
epoch 5  | loss: 24.10736|  0:00:34s
epoch 6  | loss: 18.77135|  0:00:40s
epoch 7  | loss: 16.86721|  0:00:46s
epoch 8  | loss: 14.83415|  0:00:52s
epoch 9  | loss: 14.34832|  0:00:57s
epoch 10 | loss: 13.15623|  0:01:03s
epoch 11 | loss: 12.52542|  0:01:09s
epoch 12 | loss: 12.1796 |  0:01:15s
epoch 13 | loss: 11.87858|  0:01:21s
epoch 14 | loss: 11.481  |  0:01:27s
epoch 15 | loss: 11.14743|  0:01:32s
epoch 16 | loss: 10.9185 |  0:01:38s
epoch 17 | loss: 10.7169 |  0:01:44s
epoch 18 | loss: 10.29304|  0:01:50s
epoch 19 | loss: 10.0611 |  0:01:56s
epoch 20 | loss: 9.84489 |  0:02:02s
epoch 21 | loss: 9.83032 |  0:02:07s
epoch 22 | loss: 9.76993 |  0:02:13s
epoch 23 | loss: 9.56082 |  0:02:19s
epoch 24 | loss: 9.35826 |  0:02:25s
epoch 25 | loss: 9.14902 |  0:02:31s
epoch 26 | loss: 9.0242  |  0:02:36s
